# [1] 단어-긍부정값 매핑 후 단순 연산 방식

In [3]:
import scipy as sp
import pandas as pd
import numpy as np
import scipy as sp
import re
import os
import json
import sys
import multiprocessing
from tqdm import tqdm
import random

from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Okt
from eunjeon import Mecab  # KoNLPy style mecab wrapper

import platform
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression

from tqdm import tqdm_notebook
import nltk
from nltk.tokenize import word_tokenize

import gensim
from gensim.test.utils import common_texts
from gensim.models import word2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from matplotlib import font_manager, rc

from ast import literal_eval

In [4]:
df_mapped2 = pd.read_csv('SamsungMapped2.csv')

In [5]:
df_mapped2= df_mapped2.drop(columns='Unnamed: 0')

In [6]:
df_mapped2.head(20) # 일자별 뉴스를 모아서 매핑한 데이터

,date,news,시가,고가,저가,종가,거래량,변화량
0,2017-01-01,"정성립 ""도약의 기틀 다지겠다""유창근 ""이익 내는 회사로 탈바꿈""고동진 ""자존심과 ...",0,0,0,0,0,#VALUE!
1,2017-01-02,해룡국민임대산단 내 6만 6117㎡ 매입비와 광폭판재 생산용 압연기 및 설비투자비\...,35980,36240,35880,36100,4650600,0.010526316
2,2017-01-03,수뇌부 소환 조사 임박 최순실 국정농단 사건을 수사 중인 박영수 특별검사팀의 삼성그...,36280,36620,36020,36480,7357650,-0.00877193
3,2017-01-04,ㆍ최순실 게이트 파장…조용히 치러“3년 전엔 꽉 찼는데 오늘은 절반밖에 안 모였네요...,36500,36520,36100,36160,7971750,-0.01659292
4,2017-01-05,김모 전무도 조사…삼성 지원 경위·협회 개입 여부 추궁 최송아 이보배 기자 = \'...,36060,36060,35540,35560,10967450,0.01799775
5,2017-01-06,차량용 반도체 `엑시노스`…전장분야 위상 더 높아질듯삼성전자가 미래 자동차의 핵심 ...,36180,36440,36040,36200,8880950,0.028176796
6,2017-01-07,"[앵커] 지난해 대내외적인 변수로 불확실성이 커졌지만, 새해 증시는 일단 전통적인 ...",36600,37500,36560,37220,13194900,0.000537346
7,2017-01-08,삼성전자와 LG전자가 미국에 가전공장을 건설하는 방안을 검토 중이다. 보호무역주의 ...,36600,37500,36560,37220,13194900,0.000537346
8,2017-01-09,‘삼성전자 사상 최고가’ 삼성전자가 사상 최고가를 기록해 큰 관심을 모았다.9일 삼...,36600,37500,36560,37220,13194900,0.000537346
9,2017-01-10,"트럼프 미국 대통령 당선인이 관세 부과를 무기로 기업들을 압박하면서 도요타,피아트 ...",37280,37400,37080,37240,9099800,0.02792696


In [11]:
df_news_chart = pd.read_csv('SamsungMapped6_DateNewsChart.csv', converters={'news': literal_eval}) # 일자별 뉴스토큰 매핑 데이터
df_gamsung_dict = pd.read_csv('SamsungMapped8_GamsungDictionary.csv')

In [12]:
df_news_chart= df_news_chart.drop(columns='Unnamed: 0')
df_gamsung_dict= df_token_avg.drop(columns='Unnamed: 0')

In [13]:
df_news_chart.tail(31) # 2020년 12월 매핑 데이터

,date,news,chart
1430,2020-12-01,"[사용자, 스톤, 이내, 아카데미상, 경기도, 신년, 전략, 나훈아, 가가, 서면,...",0.0251
1431,2020-12-02,"[사용자, 상무부, 스톤, 크라운, 펜싱, 고인, 이내, 아카데미상, 경기도, 타산...",0.0029
1432,2020-12-03,"[사용자, 상무부, 스톤, 사후, 고인, 이내, 경기도, 직계, 전략, 추형욱, 가...",0.0258
1433,2020-12-04,"[사용자, 김찬, 상무부, 스톤, 사후, 스텔스, 경기도, 오늘날, 조지아, 직계,...",0.0196
1434,2020-12-05,"[차익, 견인, 불문, 사용자, 표명, 김찬, 이종혁, 다운로드, 상무부, 우시, ...",-0.0165
1435,2020-12-06,"[차익, 견인, 사용자, 인증샷, 이종혁, 프로바이오틱스, 빗썸, 상무, 강화, 형...",-0.0165
1436,2020-12-07,"[사용자, 프로바이오틱스, 김찬, 다운로드, 스톤, 고인, 이내, 경기도, 전략, ...",-0.0165
1437,2020-12-08,"[사용자, 다운로드, 상무부, 사후, 이내, 경기도, 전략, 가가, 서면, 발명, ...",0.0307
1438,2020-12-09,"[사용자, 상무부, 고인, 이내, 대론, 경기도, 직계, 전략, 가가, 선언, 보험...",-0.0135
1439,2020-12-10,"[사용자, 이승기, 내빈, 고인, 이내, 경기도, 조지아, 직계, 전략, 가가, 선...",0.0069


In [14]:
df_gamsung_dict # 최종 단어별 감성사전

,token,count,chart_avg
0,변상욱,11,-0.0011
1,이창수,51,-0.0033
2,금선,10,0.0072
3,사복패션,2,-0.0002
4,임경석,1,-0.0190
...,...,...,...
92330,심지민,1,0.0019
92331,난민캠프,8,-0.0014
92332,오만,55,0.0003
92333,컨트리락,1,0.0111


In [21]:
dec_news = [news for news in df_news_chart['news'][1430:1461]] #  2020년 12월 1일 ~ 31일 뉴스 리스트
len(dec_news)

31

In [22]:
# 일자별 뉴스 토큰을 받아옴
# 사전(df_gamsung_dict)에 있는 단어가 일자별 뉴스에 있으면,
# 그 일자의 등락수치를 리스트(chart_avg_list)에 추가
# 리스트 더하고 평균 (총합 / 일자별 토큰 수)

daily_avg_list = []
for daily_news in dec_news:
    daily_len = len(daily_news)
    print('하루 토큰 수: %d'%(daily_len))
    chart_avg_list = []
    for j in range(0, 92335):
        if df_gamsung_dict['token'][j] in daily_news:
            chart_avg_list.append(df_gamsung_dict['chart_avg'][j])
    print(chart_avg_list[:20])
    print('chart_avg_list 개수: %d'%(len(chart_avg_list)))
    chart_avg_sum = round(sum(chart_avg_list),4)
    print('하루 긍부정 총합: {:.4f}'.format(chart_avg_sum))
    daily_avg = round((chart_avg_sum / daily_len),4)
    daily_avg_list.append(daily_avg)
    print('하루 긍부정 평균: {:.4f}'.format(daily_avg))
    print('============================')

daily_avg_list # 일자별 긍부정 평균

하루 토큰 수: 5075
[0.0014, 0.0013, 0.0015, -0.0009, 0.0015, 0.0061, 0.0021, 0.0017, 0.0011, 0.0023, 0.001, 0.0012, 0.0009, 0.003, 0.0133, 0.0014, 0.0054, 0.0007, 0.0016, 0.0021]
chart_avg_list 개수: 5075
하루 긍부정 총합: 9.4377
하루 긍부정 평균: 0.0019
하루 토큰 수: 5913
[0.0014, -0.0003, 0.0021, 0.0013, 0.0015, 0.0032, 0.0011, -0.0002, 0.0008, 0.0023, 0.001, 0.0012, 0.0002, 0.0009, 0.0032, 0.0024, 0.0029, 0.0014, 0.0004, 0.0054]
chart_avg_list 개수: 5913
하루 긍부정 총합: 8.7606
하루 긍부정 평균: 0.0015
하루 토큰 수: 5517
[0.0028, 0.0014, -0.0003, 0.0013, 0.0001, 0.0015, 0.0043, 0.0021, 0.0011, 0.0005, 0.0023, 0.001, 0.0012, 0.0009, 0.0001, 0.0133, -0.0006, 0.0014, 0.0025, 0.0024]
chart_avg_list 개수: 5517
하루 긍부정 총합: 10.2703
하루 긍부정 평균: 0.0019
하루 토큰 수: 5724
[0.0014, 0.0021, 0.0013, 0.0, 0.0001, -0.0009, 0.0015, 0.0062, 0.0032, 0.0011, -0.0021, 0.0023, 0.001, 0.0012, 0.0009, 0.0032, 0.0059, 0.0014, 0.0027, -0.0046]
chart_avg_list 개수: 5724
하루 긍부정 총합: 9.4943
하루 긍부정 평균: 0.0017
하루 토큰 수: 2881
[0.0014, 0.0, -0.0009, 0.0015, 0.0062, 0.0008

[0.0019,
 0.0015,
 0.0019,
 0.0017,
 0.0011,
 0.0013,
 0.001,
 0.0018,
 0.0012,
 0.0016,
 0.0015,
 0.0014,
 0.0014,
 0.0014,
 0.0014,
 0.0013,
 0.0014,
 0.0014,
 0.0014,
 0.0013,
 0.0013,
 0.0018,
 0.0021,
 0.0017,
 0.0014,
 0.0015,
 0.0014,
 0.0014,
 0.002,
 0.0014,
 0.0014]